In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from math import sqrt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [103]:
train_X = pd.read_csv('train_X.csv')
train_y = pd.read_csv('train_y.csv')
train_X.head()

,contract_id,report_date,specialization_id,contract_init_sum,contract_date,project_id,building_id,contractor_id,contract_current_sum,agg_all_contracts__g_contract__bit_da_guid__isMain__count__ALL_TIME,...,agg_ArbitrationCases__g_contractor__PlaintiffSum__sum__12_48M,agg_ArbitrationCases__g_contractor__PlaintiffSum__sum__ALL_TIME,agg_tender_proposal__g_contractor__id__ALL__countDistinct__1W,agg_tender_proposal__g_contractor__id__ALL__countDistinct__2W,agg_tender_proposal__g_contractor__id__ALL__countDistinct__4W,agg_tender_proposal__g_contractor__id__ALL__countDistinct__8W,agg_tender_proposal__g_contractor__id__ALL__countDistinct__12W,agg_tender_proposal__g_contractor__id__ALL__countDistinct__26W,agg_tender_proposal__g_contractor__id__ALL__countDistinct__52W,agg_tender_proposal__g_contractor__id__ALL__countDistinct__ALL_TIME
0,5433,2023-01-01,18,1.115267,2022-05-10 21:00:00,43,701,438,0.793952,-0.539030,...,-0.158471,-0.158473,-0.136894,-0.157822,-0.178045,-0.190966,-0.200348,-0.225724,-0.256704,-0.258454
1,6875,2023-01-01,18,1.608002,2022-08-21 21:00:00,31,268,438,2.681675,0.743673,...,-0.158471,-0.158473,-0.136894,-0.157822,-0.178045,-0.190966,-0.200348,-0.225724,-0.256704,-0.258454
2,1476,2023-01-01,18,-0.360764,2022-10-17 21:00:00,31,268,438,-0.416432,-0.539030,...,-0.158471,-0.158473,-0.136894,-0.157822,-0.178045,-0.190966,-0.200348,-0.225724,-0.256704,-0.258454
3,4469,2023-01-01,12,-0.089303,2022-04-20 21:00:00,43,697,484,-0.193827,-0.539030,...,-0.158470,-0.158471,-0.136894,-0.157822,-0.178045,-0.190966,-0.200348,-0.225724,-0.256704,-0.258454
4,1330,2023-01-01,12,-0.515778,2022-03-02 21:00:00,49,224,484,0.133446,0.743673,...,-0.158470,-0.158471,-0.136894,-0.157822,-0.178045,-0.190966,-0.200348,-0.225724,-0.256704,-0.258454


In [104]:
train_y.head()

,contract_id,report_date,default6
0,5433,2023-01-01,0
1,6875,2023-01-01,0
2,1476,2023-01-01,0
3,4469,2023-01-01,0
4,1330,2023-01-01,0


In [106]:
train_y.default6.unique()

array([0, 1])

In [107]:
train_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28831 entries, 0 to 28830
Columns: 169 entries, contract_id to agg_tender_proposal__g_contractor__id__ALL__countDistinct__ALL_TIME
dtypes: float64(162), int64(5), object(2)
memory usage: 37.2+ MB


In [108]:
quantile_lower = train_X.contract_current_sum.quantile(0.05)
quantile_higher = train_X.contract_current_sum.quantile(0.95)

train_X = train_X.dropna(subset=['agg_BoardOfDirectors__g_contractor__Name__count__ALL_TIME', 'contract_id', 'report_date', 'agg_Finance__g_contractor__Value__LongLiabilities__last__ALL_TIME', 'agg_Finance__g_contractor__Value__ShortLiabilities__last__ALL_TIME', 'contract_current_sum', 'agg_all_contracts__g_contract__bit_da_guid__isMain__count__ALL_TIME', 'agg_tender_proposal__g_contractor__id__ALL__countDistinct__ALL_TIME', 'agg_ArbitrationCases__g_contractor__PlaintiffSum__sum__ALL_TIME', 'agg_Finance__g_contractor__Value__FixedAssets__last__ALL_TIME', 'agg_scontrol__g_contractor__close_delay__defect_type_tech_supervision__mean__ALL_TIME'])
train_X = train_X[(train_X['agg_Finance__g_contractor__Value__NetProfit_y__last__ALL_TIME'] < train_X['agg_Finance__g_contractor__Value__NetProfit_y__last__ALL_TIME'].quantile(0.95)) & (train_X['agg_Finance__g_contractor__Value__NetProfit_y__last__ALL_TIME'] > train_X['agg_Finance__g_contractor__Value__NetProfit_y__last__ALL_TIME'].quantile(0.05))]
train_X = train_X[(train_X.contract_current_sum > quantile_lower) & (train_X.contract_current_sum < quantile_higher)]
train_X

,contract_id,report_date,specialization_id,contract_init_sum,contract_date,project_id,building_id,contractor_id,contract_current_sum,agg_all_contracts__g_contract__bit_da_guid__isMain__count__ALL_TIME,...,agg_ArbitrationCases__g_contractor__PlaintiffSum__sum__12_48M,agg_ArbitrationCases__g_contractor__PlaintiffSum__sum__ALL_TIME,agg_tender_proposal__g_contractor__id__ALL__countDistinct__1W,agg_tender_proposal__g_contractor__id__ALL__countDistinct__2W,agg_tender_proposal__g_contractor__id__ALL__countDistinct__4W,agg_tender_proposal__g_contractor__id__ALL__countDistinct__8W,agg_tender_proposal__g_contractor__id__ALL__countDistinct__12W,agg_tender_proposal__g_contractor__id__ALL__countDistinct__26W,agg_tender_proposal__g_contractor__id__ALL__countDistinct__52W,agg_tender_proposal__g_contractor__id__ALL__countDistinct__ALL_TIME
0,5433,2023-01-01,18,1.115267,2022-05-10 21:00:00,43,701,438,0.793952,-0.539030,...,-0.158471,-0.158473,-0.136894,-0.157822,-0.178045,-0.190966,-0.200348,-0.225724,-0.256704,-0.258454
2,1476,2023-01-01,18,-0.360764,2022-10-17 21:00:00,31,268,438,-0.416432,-0.539030,...,-0.158471,-0.158473,-0.136894,-0.157822,-0.178045,-0.190966,-0.200348,-0.225724,-0.256704,-0.258454
3,4469,2023-01-01,12,-0.089303,2022-04-20 21:00:00,43,697,484,-0.193827,-0.539030,...,-0.158470,-0.158471,-0.136894,-0.157822,-0.178045,-0.190966,-0.200348,-0.225724,-0.256704,-0.258454
4,1330,2023-01-01,12,-0.515778,2022-03-02 21:00:00,49,224,484,0.133446,0.743673,...,-0.158470,-0.158471,-0.136894,-0.157822,-0.178045,-0.190966,-0.200348,-0.225724,-0.256704,-0.258454
11,4218,2023-01-01,6,1.410171,2022-11-10 21:00:00,44,728,484,1.035780,-0.539030,...,-0.158470,-0.158471,-0.136894,-0.157822,-0.178045,-0.190966,-0.200348,-0.225724,-0.256704,-0.258454
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28814,5271,2023-07-23,10,-0.373126,2022-07-03 21:00:00,42,915,789,-0.426569,-0.539030,...,-0.158471,-0.158474,-0.136894,-0.157822,-0.178045,-0.190966,-0.200348,-0.225724,-0.256704,-0.258454
28815,3382,2023-07-23,21,-0.073025,2023-06-15 21:00:00,47,620,855,-0.180479,-0.539030,...,-0.158471,-0.158472,-0.136894,-0.157822,-0.178045,-0.190966,-0.200348,-0.225724,-0.256704,-0.258454
28819,5465,2023-07-23,21,0.090307,2023-05-04 21:00:00,47,621,855,-0.046542,-0.539030,...,-0.158471,-0.158472,-0.136894,-0.157822,-0.178045,-0.190966,-0.200348,-0.225724,-0.256704,-0.258454
28820,6647,2023-07-23,17,1.009716,2022-11-15 21:00:00,45,533,855,0.610944,0.743673,...,-0.158471,-0.158472,-0.136894,-0.157822,-0.178045,-0.190966,-0.200348,-0.225724,-0.256704,-0.258454


In [109]:
train_X.isna().sum()

contract_id                                                            0
report_date                                                            0
specialization_id                                                      0
contract_init_sum                                                      0
contract_date                                                          0
                                                                      ..
agg_tender_proposal__g_contractor__id__ALL__countDistinct__8W          0
agg_tender_proposal__g_contractor__id__ALL__countDistinct__12W         0
agg_tender_proposal__g_contractor__id__ALL__countDistinct__26W         0
agg_tender_proposal__g_contractor__id__ALL__countDistinct__52W         0
agg_tender_proposal__g_contractor__id__ALL__countDistinct__ALL_TIME    0
Length: 169, dtype: int64

In [110]:
f = train_X.isna().sum().to_frame("Nans").reset_index()
s = 0
nan_cols = []
for index, value in enumerate(train_X.isna().sum()):
    if value != 0:
        s += 1
        print(f.loc[index, 'index'], value)
        nan_cols.append(f.loc[index, 'index'])

print(s)

agg_all_contracts__g_contract__abs_change_price_last_ds__isMain__last__ALL_TIME 9112
agg_all_contracts__g_contract__abs_change_price_last_ds__isMain__mean__ALL_TIME 9112
agg_all_contracts__g_contract__rel_change_price_last_ds__isMain__last__ALL_TIME 10154
agg_all_contracts__g_contract__rel_change_price_last_ds__isMain__mean__ALL_TIME 10154
agg_sroomer__g_contractor__sroomer_id__count__3M 7263
agg_sroomer__g_contractor__sroomer_id__count__6M 7263
agg_sroomer__g_contractor__sroomer_id__count__12M 7263
agg_sroomer__g_contractor__sroomer_id__count__ALL_TIME 7263
agg_FinanceAndTaxesFTS__g_contractor__TaxArrearsSum__last__ALL_TIME 7727
agg_FinanceAndTaxesFTS__g_contractor__TaxPenaltiesSum__last__ALL_TIME 10997
10


In [111]:
f[f['Nans'] != 0]

,index,Nans
10,agg_all_contracts__g_contract__abs_change_pric...,9112
11,agg_all_contracts__g_contract__abs_change_pric...,9112
12,agg_all_contracts__g_contract__rel_change_pric...,10154
13,agg_all_contracts__g_contract__rel_change_pric...,10154
119,agg_sroomer__g_contractor__sroomer_id__count__3M,7263
120,agg_sroomer__g_contractor__sroomer_id__count__6M,7263
121,agg_sroomer__g_contractor__sroomer_id__count__12M,7263
122,agg_sroomer__g_contractor__sroomer_id__count__...,7263
148,agg_FinanceAndTaxesFTS__g_contractor__TaxArrea...,7727
149,agg_FinanceAndTaxesFTS__g_contractor__TaxPenal...,10997


In [112]:
train_X = train_X.drop(columns=nan_cols).reset_index().drop(columns='index')
df = pd.merge(train_X, train_y, how='inner', on=['contract_id', 'report_date'])
dates = np.array(df.dtypes[df.dtypes == 'object'].index.tolist())
df[dates[0]] = pd.to_datetime(df[dates[0]])
df[dates[1]] = pd.to_datetime(df[dates[1]])
X = df[train_X.columns]
y = df[train_y.columns]

In [11]:
y.dtypes

contract_id             int64
report_date    datetime64[ns]
default6                int64
dtype: object

In [12]:
X.dtypes

contract_id                                                                     int64
report_date                                                            datetime64[ns]
specialization_id                                                               int64
contract_init_sum                                                             float64
contract_date                                                          datetime64[ns]
                                                                            ...      
agg_tender_proposal__g_contractor__id__ALL__countDistinct__8W                 float64
agg_tender_proposal__g_contractor__id__ALL__countDistinct__12W                float64
agg_tender_proposal__g_contractor__id__ALL__countDistinct__26W                float64
agg_tender_proposal__g_contractor__id__ALL__countDistinct__52W                float64
agg_tender_proposal__g_contractor__id__ALL__countDistinct__ALL_TIME           float64
Length: 159, dtype: object

In [113]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [114]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

In [115]:
X.to_csv('X_train.csv')
y.to_csv('y_train.csv')

In [116]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier

# Sample DataFrame creation
# df = pd.read_csv('your_data.csv')

# Convert datetime columns to numeric features
def preprocess_datetime(df):
    # Ensure columns are in datetime format
    df['report_date'] = pd.to_datetime(df['report_date'])
    df['contract_date'] = pd.to_datetime(df['contract_date'])

    # Extract year, month, day, weekday, etc.
    df['report_year'] = df['report_date'].dt.year
    df['report_month'] = df['report_date'].dt.month
    df['report_day'] = df['report_date'].dt.day
    df['report_weekday'] = df['report_date'].dt.weekday

    df['contract_year'] = df['contract_date'].dt.year
    df['contract_month'] = df['contract_date'].dt.month
    df['contract_day'] = df['contract_date'].dt.day
    df['contract_weekday'] = df['contract_date'].dt.weekday

    # Drop original datetime columns if no longer needed
    # df = df.drop(columns=['report_date', 'contract_date'])

    return df

# Apply preprocessing
df = preprocess_datetime(df)

# Separate features and target
X = df.drop(columns=['default6', 'contract_date', 'report_date'])
y = df['default6']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the XGBoost model
xgboost = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Define the parameter grid for GridSearchCV
parameters = {
    'n_estimators': [500],
    'learning_rate': [0.1],
    'max_depth': [7],
    'min_child_weight': [1],
    'gamma': [0],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8]
}

# Set up the GridSearchCV
clf = GridSearchCV(estimator=xgboost, param_grid=parameters, scoring='accuracy', cv=5, n_jobs=-1)

# Fit the model
clf.fit(X_train, y_train)

# Print the best parameters and best score
print("Best Parameters:", clf.best_params_)
print("Best CV Score:", clf.best_score_)

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
print("Test Set Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


/home/clodis/Projects/Python/Plane_aichal/.conda/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [14:38:12] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/clodis/Projects/Python/Plane_aichal/.conda/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [14:38:12] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/clodis/Projects/Python/Plane_aichal/.conda/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [14:38:12] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/clodis/Projects/Python/Plane_aichal/.conda/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [14:38:12] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Best Parameters: {'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 7, 'min_child_weight': 1, 'n_estimators': 500, 'subsample': 0.8}
Best CV Score: 0.9894510523539595
Test Set Accuracy: 0.9888535031847133

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99      2075
           1       0.97      0.97      0.97       437

    accuracy                           0.99      2512
   macro avg       0.98      0.98      0.98      2512
weighted avg       0.99      0.99      0.99      2512



In [117]:
from sklearn .metrics import roc_auc_score
auc = roc_auc_score(y_test, y_pred)
auc * 2 - 1

np.float64(0.9594099969672742)

In [118]:
test = pd.read_csv('test1_X.csv')
test = preprocess_datetime(test)
test[X_train.columns]
test_X = test
test_y = test[['contract_id', 'report_date']]

In [119]:
subm = pd.read_csv('submission.csv')

In [120]:
subm

,contract_id,report_date,score
0,3029,2023-07-30,0.002356
1,4350,2023-07-30,0.011442
2,1095,2023-07-30,0.039829
3,2634,2023-07-30,0.009016
4,6535,2023-07-30,0.000732
...,...,...,...
7327,4740,2023-09-24,0.001761
7328,2689,2023-09-24,0.052340
7329,7135,2023-09-24,0.106463
7330,4164,2023-09-24,0.001905


In [95]:
test_y

,contract_id,report_date,score
0,3029,2023-07-30,0
1,4350,2023-07-30,0
2,1095,2023-07-30,0
3,2634,2023-07-30,0
4,6535,2023-07-30,0
...,...,...,...
7327,4740,2023-09-24,0
7328,2689,2023-09-24,0
7329,7135,2023-09-24,0
7330,4164,2023-09-24,0


In [124]:
def preprocess_datetime(df):
    # Ensure columns are in datetime format
    df['report_date'] = pd.to_datetime(df['report_date'])
    df['contract_date'] = pd.to_datetime(df['contract_date'])

    # Extract year, month, day, weekday, etc.
    df['report_year'] = df['report_date'].dt.year
    df['report_month'] = df['report_date'].dt.month
    df['report_day'] = df['report_date'].dt.day
    df['report_weekday'] = df['report_date'].dt.weekday

    df['contract_year'] = df['contract_date'].dt.year
    df['contract_month'] = df['contract_date'].dt.month
    df['contract_day'] = df['contract_date'].dt.day
    df['contract_weekday'] = df['contract_date'].dt.weekday

    # Drop original datetime columns if no longer needed
    df = df.drop(columns=['report_date', 'contract_date'])

    return df

In [126]:
# ans = clf.predict(test_X[X_train.columns].to_numpy()[])
ans = clf.predict(preprocess_datetime(test_X[train_X.columns]))

/tmp/ipykernel_4163/2316066732.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['report_date'] = pd.to_datetime(df['report_date'])
/tmp/ipykernel_4163/2316066732.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['contract_date'] = pd.to_datetime(df['contract_date'])
/tmp/ipykernel_4163/2316066732.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

In [128]:
# If you're testing classification
subm.score = ans

In [130]:
subm.score.value_counts()

score
0    6745
1     587
Name: count, dtype: int64

In [142]:
# For uploading
ans = clf.predict_proba(preprocess_datetime(test_X[train_X.columns]))
subm.score = ans

/tmp/ipykernel_4163/2316066732.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['report_date'] = pd.to_datetime(df['report_date'])
/tmp/ipykernel_4163/2316066732.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['contract_date'] = pd.to_datetime(df['contract_date'])
/tmp/ipykernel_4163/2316066732.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

In [149]:
subm.score = 1 - subm.score
subm.to_csv('test_subm.csv', index = False)

In [150]:
subm

,contract_id,report_date,score
0,3029,2023-07-30,0.004177
1,4350,2023-07-30,0.052343
2,1095,2023-07-30,0.030637
3,2634,2023-07-30,0.005377
4,6535,2023-07-30,0.000963
...,...,...,...
7327,4740,2023-09-24,0.001294
7328,2689,2023-09-24,0.040765
7329,7135,2023-09-24,0.002304
7330,4164,2023-09-24,0.025056


In [151]:
subm.max()

contract_id          7338
report_date    2023-09-24
score            0.999615
dtype: object